In [2]:
from selenium_profiles.webdriver import Chrome
from selenium_profiles.profiles import profiles
from selenium.webdriver.common.by import By
from seleniumwire import webdriver
import time
import pandas as pd

In [7]:
profile = profiles.Windows() # or .Android
options = webdriver.ChromeOptions()
# options.add_argument("--headless=new")
driver = Chrome(profile, options=options,
                uc_driver=False
                )

driver.get("https://store.steampowered.com/search/?filter=topsellers")
for i in range(2):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)
start = 0
end = 5
# time.sleep(5)
show_select = driver.find_element(By.XPATH, '//*[@id="advsearchform"]/div[1]/div/div[1]')
# show_select.click()

# games = driver.find_elements(By.XPATH, f'//tbody/tr[position()>{start} and position()<{end}]')
games = driver.find_elements(By.XPATH, f'//*[@id="search_resultsRows"]/a[position()>{start} and position()<{end}]')

game_links = []
game_names = []
links = []
for game in games:
    game_link = game.get_attribute('href')
    game_links.append(game_link)


for link in game_links:
    driver.get(link)
    time.sleep(3)
    try:
        # game.find_element(By.XPATH, '//*[@id="app_agegate"]/div[1]/div[2]/div').get_attribute('href')
        day = driver.find_element(By.ID, 'ageDay')
        day.click()
        day_menu = driver.find_element(By.XPATH, "//option[@value='4']")
        day_menu.click()
        month = driver.find_element(By.ID, 'ageMonth')
        month.click()
        month_menu = driver.find_element(By.XPATH, "//option[@value='April']")
        month_menu.click()
        year = driver.find_element(By.ID, 'ageYear')
        year.click()
        year_menu = driver.find_element(By.XPATH, "//option[@value='2000']")
        year_menu.click()
        view_page = driver.find_element(By.XPATH, '//*[@id="view_product_page_btn"]/span')
        view_page.click()
        print('Age gate passed, continuing...')
        time.sleep(3)

    except:
        print('No age gate, continuing...')
    driver.execute_script("""
    for(let script of document.querySelectorAll('script')) script.remove()
    """)
    try:
        game_name = driver.find_element(By.XPATH, '//*[@id="appHubAppName"]').text
        game_names.append(game_name)
        links.append(link)
        print(game_name)
    except:
        print('No game name found')
    

df = pd.DataFrame({'Game Name': game_names, 'Link': links})
df.to_csv('steam_games.csv', index=False)
print(len(game_links))
print(game_links)
driver.quit()

No age gate, continuing...
Palworld
No age gate, continuing...
Granblue Fantasy: Relink
No age gate, continuing...
Enshrouded
No age gate, continuing...
Counter-Strike 2
4
['https://store.steampowered.com/app/1623730/Palworld/?snr=1_7_7_7000_150_1', 'https://store.steampowered.com/app/881020/Granblue_Fantasy_Relink/?snr=1_7_7_7000_150_1', 'https://store.steampowered.com/app/1203620/Enshrouded/?snr=1_7_7_7000_150_1', 'https://store.steampowered.com/app/730/CounterStrike_2/?snr=1_7_7_7000_150_1']
